import library

In [26]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc
from sklearn.preprocessing import StandardScaler, LabelEncoder

Load data

In [ ]:
train_data = pd.read_csv('./DataMining/input/test.csv')
test_data = pd.read_csv('./DataMining/input/test.csv')
print("Train Data Info:")
print(train_data.info())
print("Train Data Head:")
print(train_data.head())

print("Test Data Info:")
print(test_data.info())
print("Test Data Head:")
print(test_data.head())


Xử lý duplicate bản ghi trùng lặp

In [3]:
duplicate_train = train_data.duplicated()
print("Number of duplicate train rows", duplicate_train.sum())
duplicate_test = test_data.duplicated()
print("Number of duplicate test rows", duplicate_test.sum())
train_data.drop_duplicates()
test_data.drop_duplicates()

Number of duplicate train rows 0
Number of duplicate test rows 0


,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,19556,Female,Loyal Customer,52,Business travel,Eco,160,5,4,...,5,5,5,5,2,5,5,50,44.0,satisfied
1,1,90035,Female,Loyal Customer,36,Business travel,Business,2863,1,1,...,4,4,4,4,3,4,5,0,0.0,satisfied
2,2,12360,Male,disloyal Customer,20,Business travel,Eco,192,2,0,...,2,4,1,3,2,2,2,0,0.0,neutral or dissatisfied
3,3,77959,Male,Loyal Customer,44,Business travel,Business,3377,0,0,...,1,1,1,1,3,1,4,0,6.0,satisfied
4,4,36875,Female,Loyal Customer,49,Business travel,Eco,1182,2,3,...,2,2,2,2,4,2,4,0,20.0,satisfied
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25971,25971,78463,Male,disloyal Customer,34,Business travel,Business,526,3,3,...,4,3,2,4,4,5,4,0,0.0,neutral or dissatisfied
25972,25972,71167,Male,Loyal Customer,23,Business travel,Business,646,4,4,...,4,4,5,5,5,5,4,0,0.0,satisfied
25973,25973,37675,Female,Loyal Customer,17,Personal Travel,Eco,828,2,5,...,2,4,3,4,5,4,2,0,0.0,neutral or dissatisfied
25974,25974,90086,Male,Loyal Customer,14,Business travel,Business,1127,3,3,...,4,3,2,5,4,5,4,0,0.0,satisfied


Xử lý giá trị thiếu missing

In [4]:
missing_train = train_data.isnull().sum()
missing_test = test_data.isnull().sum()
print("Missing values in train data:\n", missing_train)
print("Missing values in test data:\n", missing_test)

Missing values in train data:
 Unnamed: 0                            0
id                                    0
Gender                                0
Customer Type                         0
Age                                   0
Type of Travel                        0
Class                                 0
Flight Distance                       0
Inflight wifi service                 0
Departure/Arrival time convenient     0
Ease of Online booking                0
Gate location                         0
Food and drink                        0
Online boarding                       0
Seat comfort                          0
Inflight entertainment                0
On-board service                      0
Leg room service                      0
Baggage handling                      0
Checkin service                       0
Inflight service                      0
Cleanliness                           0
Departure Delay in Minutes            0
Arrival Delay in Minutes             83
satisfact

loại bỏ các hàng chứa giá trị thiếu

In [5]:
train_data = train_data.dropna()
test_data = test_data.dropna()
# thay gia tri missing bang gia tri trung binh cua cot do
# train_data = train_data.fillna(train_data.mean())
# test_data = test_data.fillna(test_data.mean())

Chia thành tập huấn luyện và kiểm tra nội bộ

In [18]:
x_train_full = train_data.drop('satisfaction', axis=1)
y_train_full = train_data['satisfaction']
x_test = test_data.drop('satisfaction', axis=1)
y_test = test_data['satisfaction']
# Chia thành tập huấn luyện và kiểm tra nội bộ
x_train, x_val, y_train, y_val = train_test_split(x_train_full, y_train_full, test_size=0.2, random_state=42)

Mã hóa các giá trị chuỗi

In [17]:
label_encoders = {}
for column in train_data.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    train_data[column] = le.fit_transform(train_data[column])
    test_data[column] = le.transform(test_data[column])
    label_encoders[column] = le

chuẩn hóa dữ liệu

In [20]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)
x_test = scaler.transform(x_test)

Huấn luyện mô hình

In [22]:
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(x_train, y_train)

SVC(kernel='linear', random_state=42)

dự đoán, đánh giá

In [25]:
# Dự đoán
y_val_pred = svm_model.predict(x_val)
y_test_pred = svm_model.predict(x_test)

# Đánh giá
print("Validation Accuracy:", accuracy_score(y_val, y_val_pred))
print("Test Accuracy:", accuracy_score(y_test, y_test_pred))
print("Validation Report:\n", classification_report(y_val, y_val_pred))
print("Test Report:\n", classification_report(y_test, y_test_pred))

# Thêm ma trận nhầm lẫn
print("Validation Confusion Matrix:\n", confusion_matrix(y_val, y_val_pred))
print("Test Confusion Matrix:\n", confusion_matrix(y_test, y_test_pred))

Validation Accuracy: 0.8773894574242131
Test Accuracy: 0.8750627582744371
Validation Report:
               precision    recall  f1-score   support

           0       0.87      0.91      0.89      2890
           1       0.88      0.83      0.86      2289

    accuracy                           0.88      5179
   macro avg       0.88      0.87      0.87      5179
weighted avg       0.88      0.88      0.88      5179

Test Report:
               precision    recall  f1-score   support

           0       0.87      0.91      0.89     14528
           1       0.88      0.83      0.85     11365

    accuracy                           0.88     25893
   macro avg       0.88      0.87      0.87     25893
weighted avg       0.88      0.88      0.87     25893

Validation Confusion Matrix:
 [[2642  248]
 [ 387 1902]]
Test Confusion Matrix:
 [[13255  1273]
 [ 1962  9403]]
